# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 20% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [ ]:
def read_Energy_df():
    #Read the Energy file in and fix it up the way they want it
    Energy = pd.read_excel('assets/Energy Indicators.xls', header = 16, skipfooter=38)
    Energy.drop(0,inplace=True)
    Energy.rename(columns={'Unnamed: 1':'Country','Unnamed: 2': 'Country 2', 
                           'Renewable Electricity Production': '% Renewable', 
                           'Energy Supply per capita': 'Energy Supply per Capita'}, inplace=True)
    Energy.drop(columns=['Unnamed: 0', 'Country 2'],inplace=True)
    #replace ... with NaN
    Energy.replace('...', np.nan, inplace = True)
    #Convert Energy Supply from petrojoules to gigajoules
    Energy['Energy Supply'] = Energy['Energy Supply'].apply(lambda x:x*1000000)
    #Change names of specific countries
    Energy.replace({'Republic of Korea': 'South Korea', 
                   'United States of America': 'United States', 
                  'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom', 
                  'China, Hong Kong Special Administrative Region': 'Hong Kong'}, inplace=True)
    #Get rid of parentheses in country names
    Energy['Country'] = Energy['Country'].str.replace(r"\s\(.*\)","")
    #if you want to set the index to country name 
    Energy.set_index('Country', inplace=True) 
    return Energy

read_Energy_df()

In [ ]:
def read_GDP_df():
    #Read in the world_bank file and skip header
    GDP = pd.read_csv('assets/world_bank.csv', header = 4)
    #Change names of specific countries
    GDP.replace({'Korea, Rep.': 'South Korea', 
                'Iran, Islamic Rep.': 'Iran', 
                'Hong Kong SAR, China': 'Hong Kong'}, inplace = True)
    #delete info from before 2006
    gdp_headers = list(GDP.columns.values)
    del gdp_headers[1:50]
    GDP = GDP[gdp_headers]
    #Rename "Country Name" to "Country"
    GDP.rename(columns={'Country Name':'Country'}, inplace=True)
    #if you want to set the index to country name 
    GDP.set_index('Country', inplace=True)
    return GDP

read_GDP_df()

In [ ]:
def read_ScimEn_df():
    #Read in the country rank for energy engineering data 
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    #if you want to set the index to country name 
    ScimEn.set_index('Country', inplace=True)
    return ScimEn

read_ScimEn_df()

In [ ]:
def answer_one():
    Energy = read_Energy_df()
    GDP = read_GDP_df()
    ScimEn = read_ScimEn_df()
    #Merge the DataFrames
    df = pd.merge(ScimEn,pd.merge(Energy, GDP, how = 'inner', on='Country'), how = 'inner', on='Country')
    Top15=df[:15] #cut off any countries whose rank is < 15
    return Top15

answer_one()

In [ ]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [ ]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_two():
    Energy = read_Energy_df()
    GDP = read_GDP_df()
    ScimEn = read_ScimEn_df()
    
    outer_df = pd.merge(Energy, pd.merge(GDP, ScimEn, how='outer', on='Country'), how='outer', on='Country')
    inner_df = pd.merge(Energy, pd.merge(GDP, ScimEn, how='inner', on='Country'), how='inner', on='Country')
    
    numentries_outer = len(outer_df.index)
    numentries_inner = len(inner_df.index)
    
    numentrieslost = numentries_outer - numentries_inner
    return numentrieslost

answer_two()

In [ ]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_three():
    Energy = read_Energy_df()
    GDP = read_GDP_df()
    df = pd.merge(GDP, Energy, how='inner', on='Country')
    df['avgGDP'] = df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1,skipna=True)
    df.sort_values('avgGDP', ascending=False, inplace=True, na_position='last')
    df_top15=df[:15]
    avgGDP = pd.Series(df_top15['avgGDP'], index= df_top15.index)
    #confused if there's an easier way to just get the countries out? 
    return avgGDP

answer_three()

In [ ]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_four():
    Energy = read_Energy_df()
    GDP = read_GDP_df()
    df = pd.merge(GDP, Energy, how='inner', on='Country')
    df['avgGDP'] = df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1,skipna=True)
    df.sort_values('avgGDP', ascending=False, inplace=True, na_position='last')
    
    #get difference in GDP over 10 years
    df['GDPdiff'] = df['2015']-df['2006']
    
    #find the one that is number 6 with iloc[5]
    sixth = df['GDPdiff'].iloc[5]
    return sixth

answer_four()

In [ ]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [ ]:
def answer_five():
    Energy = read_Energy_df()

    avg_energysupplypercapita = Energy['Energy Supply per Capita'].mean()
    return avg_energysupplypercapita

answer_five()

In [ ]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    Energy = read_Energy_df()
    max_energy_entry= Energy.nlargest(1,'% Renewable') #.idmax finds the index of the max entry
    max_pct_renew = (max_energy_entry.index[0], max_energy_entry['% Renewable'][0])
    return max_pct_renew

answer_six()

In [ ]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    ScimEn = read_ScimEn_df()
    ScimEn['Citations Ratio'] = ScimEn['Self-citations']/ScimEn['Citations']
    max_citations_ratio_entry = ScimEn.nlargest(1,'Citations Ratio')
    max_cit_ratio = (max_citations_ratio_entry.index[0], max_citations_ratio_entry['Citations Ratio'][0])
    return max_cit_ratio

answer_seven()

In [ ]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [ ]:
def answer_eight():
    Energy = read_Energy_df()
    Energy['Pop Est']=Energy['Energy Supply']/Energy['Energy Supply per Capita']
    Energy.sort_values('Pop Est', ascending=False, inplace=True)
    
    third_most_populous = Energy.index[2]
    return third_most_populous

answer_eight()

In [ ]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    Energy = read_Energy_df()
    ScimEn = read_ScimEn_df()
    
    Energy['Pop Est']=Energy['Energy Supply']/Energy['Energy Supply per Capita']
    Energy.sort_values('Pop Est', ascending=False, inplace=True)
    df = pd.merge(ScimEn, Energy, how = 'outer', on = 'Country')
    df['Citable Documents per Capita'] = df['Citable documents']/df['Pop Est']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    newdf = df[['Energy Supply per Capita','Citable Documents per Capita']].dropna()
    corr, pval = stats.pearsonr(newdf['Citable Documents per Capita'], newdf['Energy Supply per Capita'])
    return corr

answer_nine()

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
plot9()

In [ ]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    print(median)
    
    #for value in Top15['% Renewable']:
    #    if(value>=median):
    #        Top15['HighRenew'] = 1 #need to specify row here  
    #    else:               
    #        Top15['HighRenew'] = 0
    #for reasons unkonwn this loop didn't work? 
    
    Top15['HighRenew']=Top15['% Renewable'].gt(median).astype(int) #cast the boolean to an integer
    HighRenew = pd.Series(Top15['HighRenew'], index=Top15.index)
    return HighRenew

answer_ten()

In [ ]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    Top15['Pop Est']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15['Continent']=pd.Series(ContinentDict)
    
    continentdf = Top15.groupby('Continent').agg({'Pop Est': (np.size,np.sum,np.nanmean,np.nanstd)})
    continentdf=continentdf.droplevel(level=0, axis=1)
    continentdf.rename(columns={'nanmean':'mean', 'nanstd':'std'}, inplace=True)
    return continentdf

answer_eleven()

In [ ]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    
    out= pd.cut(Top15['% Renewable'].values, bins = 5) #.values returns a numpy representation of the dataframe
    con = Top15.index.to_series().map(ContinentDict).values #returns an array of continents in the same order as the index of countries
    
    Top15['Bin']= out 
    Top15['Continent'] = con
    
    result = Top15.groupby(['Continent', 'Bin']).size() #this returns a series with multiindex in order with data of size
    
    #Top15.reset_index(inplace=True) #turns index into 0, 1, 2, etc. 
    #Top15.index = pd.MultiIndex.from_arrays([con, out]) #creates the Multiindex in the given order
    #Top15.sort_index(inplace=True) #sorts the index
    
    
    #need to make this a series still 
    return result
    
answer_twelve()



In [ ]:
Top15 = answer_one()
Top15['Letter'] = Top15.apply(lambda r: r.name[0], axis=1)

Top15.groupby('Letter') #returns nothing, but it's a "group object" -- collection of keys and groups

for letter, group in Top15.groupby('Letter'):
    print(letter)
    
group

In [ ]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [ ]:
#pd.options.display.float_format = '{:20,.2f}'.format #globally suppresses scientific notation

def answer_thirteen():
    Top15 = answer_one()
    Top15['Pop Est']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    PopEst = Top15['Pop Est']
    PopEst = PopEst.apply('{:,}'.format) #also turns it to a string
    return PopEst

answer_thirteen()

In [ ]:
'{:,}'.format(2000)

In [ ]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")